In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import dill
import tensorflow as tf
from tensorflow import keras
import random
from sklearn.model_selection import GroupKFold

! cp /kaggle/input/jmarket-keras-starter/*model.hdf5 ./
! cp /kaggle/input/jmarket-keras-starter/val_model.hdf5 ./best_val_model.hdf5

In [ ]:
np.random.seed(1230)
random.seed(1230)

We converted the dataset to the **.feather** format in this [Notebook](https://www.kaggle.com/tchaye59/jane-street-market-dataset-to-feather). It is faster to load and support all build in pandas functionality.

In [ ]:
train_df = pd.read_feather('/kaggle/input/jane-street-market-dataset-to-feather/train.feather')
feature_df = pd.read_feather('/kaggle/input/jane-street-market-dataset-to-feather/features.feather')
feature_df.set_index('feature',inplace=True)

## Define The target

In [ ]:
train_df['target'] = (train_df.resp>0).astype(np.int32)

## Prepare the dataset

In [ ]:
# Use all columns in the dataframe as a feature  except these
ignore_columns = ['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp','ts_id','target']
feature_cols = [col for col in train_df.columns if col not in ignore_columns]

# We will need it during submission
dill.dump(feature_cols,open('feature_cols.dill','wb'))

In [ ]:
# We will work with NumPy arrays
train_df.fillna(0,inplace=True)
train_df = train_df.sample(frac=1.,random_state=0)
y = train_df.target.values 
X = train_df[feature_cols].values.astype(np.float32)
dates = train_df[['date',]].values
del train_df

In [ ]:
sss = GroupKFold(n_splits=5,)
sss.get_n_splits(X, y,dates)

In [ ]:
for train_index, test_index in sss.split(X, y,dates):
    print("TRAIN:", train_index.shape, "TEST:", test_index.shape)
    X_train,X_test,y_train,y_test = X[train_index],X[test_index],y[train_index],y[test_index]
    break

In [ ]:
filepath="val_model{val_acc}.hdf5"
callbacks_list = [
    keras.callbacks.ModelCheckpoint(filepath, 
                                    verbose=1,
                                    monitor='val_acc', 
                                    save_best_only=True, 
                                    mode='max'),
]
batch_size = 2**12
batch_size

# Build the model

In [ ]:
model = keras.Sequential([
    keras.layers.BatchNormalization(input_shape=(len(feature_cols),)),
    
    keras.layers.Dense(512),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(512),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(512),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(512),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.2),
    
    
    keras.layers.Dense(1,activation='sigmoid'),
])

if os.path.exists('./last_model.hdf5'):
    print('Loading...')
    model = keras.models.load_model('./last_model.hdf5')
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(0.001),
              metrics=['acc',]
             )
model.summary()

## Train

In [ ]:
history = model.fit(X_train,y_train,
                    batch_size=batch_size,
                    validation_data=(X_test,y_test,),
                    epochs=100,
                    callbacks=callbacks_list)
model.save('last_model.hdf5')

In [ ]:
history_df = pd.DataFrame(history.history)
#history_df.loss = MinMaxScaler().fit_transform(history_df[['loss',]])
#history_df.val_loss = MinMaxScaler().fit_transform(history_df[['val_loss',]])
history_df.plot()

It is better to separate submission and training:

## [Check the submission notebook](https://www.kaggle.com/tchaye59/jmarket-keras-starter-submit)